In [ ]:
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
import tf_keras
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import re
import sys
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.callbacks import EarlyStopping
from scipy.special import softmax
import numpy as np
from sklearn.preprocessing import MinMaxScaler
!pip install datasets
import datasets
from datasets import load_dataset
from transformers import BatchEncoding
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (1)
from google.colab import files
uploaded = files.upload()

In [ ]:
#RUN ONLY IF YOU WANT THE Coronavirus NLP tweets DATASET (2)
train_df=pd.read_csv('/content/Corona_NLP_train.csv', encoding='ISO-8859-1')
test_df=pd.read_csv('/content/Corona_NLP_test.csv', encoding='ISO-8859-1')
train_df=train_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
test_df=test_df.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1)
train_df.drop_duplicates(subset='OriginalTweet',inplace=True)
test_df.drop_duplicates(subset='OriginalTweet',inplace=True)
df=pd.concat([train_df, test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['Sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['Sentiment'], random_state=42)
mapping = {'Extremely Negative': 0, 'Negative': 1, 'Neutral': 2, 'Positive':3, 'Extremely Positive':4}
train_df['Sentiment']=train_df['Sentiment'].replace(mapping)
val_df['Sentiment']=val_df['Sentiment'].replace(mapping)
test_df['Sentiment']=test_df['Sentiment'].replace(mapping)
train_df['Sentiment']=train_df['Sentiment'].astype('category')
val_df['Sentiment']=val_df['Sentiment'].astype('category')
test_df['Sentiment']=test_df['Sentiment'].astype('category')
train_df, unlab_df= train_test_split(train_df, test_size=0.42, stratify=train_df['label'], random_state=42)
unlab_df['Sentiment']=unlab_df['Sentiment'].cat.add_categories(5)
unlab_df['Sentiment']=5
#For the first split, put the frac value below to be equal to 0.0051
#For the second split, put the frac value below to be equal to 0.0255
#For the third split, put the frac value below to be equal to 0.051
#For the fourth split, put the frac value below to be equal to 0.2545
train_df=train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.2545))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE IMDB Movie Review DATASET
cwd=os.getcwd()
data=tf.keras.utils.get_file(fname ="aclImdb.tar.gz", origin="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", cache_dir= cwd, extract = True)
data_path=os.path.join(os.path.dirname(data), 'aclImdb')
os.listdir(data_path)
train_dir=os.path.join(data_path, 'train')
for file in os.listdir(train_dir):
    file_path=os.path.join(train_dir, file)
    if os.path.isfile(file_path):
        with open(file_path) as f:
            first_line=f.readline().strip()
            print(file + ":" + first_line)
def load_dataset(dir, typeOfData):
    dictnry = {"review": [], "sentiment": []}
    if typeOfData == 'labelled':
        pos_path=os.path.join(dir, 'pos')
        neg_path=os.path.join(dir, 'neg')
        for file in os.listdir(pos_path):
            file_path=os.path.join(pos_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(1)
        for file in os.listdir(neg_path):
            file_path=os.path.join(neg_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(0)
    elif typeOfData == 'unsup':
        unsup_path=os.path.join (dir, 'unsup')
        for file in os.listdir(unsup_path):
            file_path=os.path.join(unsup_path, file)
            with open(file_path, 'r') as f:
                dictnry["review"].append(f.read())
                dictnry["sentiment"].append(2)
    return dictnry
Train_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'labelled'))
Unlab_df=pd.DataFrame.from_dict(load_dataset(train_dir, 'unsup'))
test_dir=os.path.join(data_path, 'test')
Test_df=pd.DataFrame.from_dict(load_dataset(test_dir, 'labelled'))
df=pd.concat([Train_df, Test_df], axis=0)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['sentiment'], random_state=42)
train_df, unlab_df= train_test_split(train_df, test_size=0.375, stratify=train_df['sentiment'], random_state=42)
unlab_df.loc[:, 'sentiment']=2
#For the first split, put the frac value from the train_df below to be equal to 0.004 for about 100 labeled
#For the second split, put the frac value from the train_df below to be equal to 0.012 for about 300 labeled
#For the third split, put the frac value from the train_df below to be equal to 0.04 for about 1000 labeled
#For the fourth split, put the frac value from the train_df below to be equal to 0.2 for about 5000 labeled
train_df=train_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=0.2))
val_df=val_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
test_df=test_df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(frac=1))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (1)
sst2_dataset = load_dataset("glue", "sst2")
train_dataset = sst2_dataset["train"]
validation_dataset = sst2_dataset["validation"]
test_dataset = sst2_dataset["test"]
train_df=pd.DataFrame(train_dataset.to_pandas())
val_df=pd.DataFrame(validation_dataset.to_pandas())
test_df=pd.DataFrame(test_dataset.to_pandas())
df=pd.concat([train_df, val_df], ignore_index=True)
df.drop(columns=['idx'], inplace=True)
print(df)

In [ ]:
#RUN ONLY IF YOU WANT THE SST-2 DATASET (2)
train_df, test_df=train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df=train_test_split(test_df, test_size=0.5, stratify=test_df['label'], random_state=42)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df, unlab_df= train_test_split(train_df, test_size=0.4, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=2
print(train_df)
print(unlab_df)
print(val_df)
print(test_df)
#For the first split, put the frac value below to be equal to 0.0031 for about 100 labeled samples
#For the second split, put the frac value below to be equal to 0.01 for about 300 labeled samples
#For the third split, put the frac value below to be equal to 0.0153 for about 500 labeled samples
#For the fourth split, put the frac value below to be equal to 0.031 for about 1000 labeled samples
#For the fifth split put the frac value below to be equal to 0.0765 for about 2500 labeled samples
#For the sixth split, put the frac value below to be equal to 0.153 for about 5000 labeled samples
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.153))
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
#RUN ONLY IF YOU WANT SST-5 DATASET
dataset=load_dataset("SetFit/sst5")
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']
train_df=pd.DataFrame(train_dataset)
train_df=train_df.drop(columns=['label_text'])
val_df=pd.DataFrame(val_dataset)
val_df=val_df.drop(columns=['label_text'])
test_df=pd.DataFrame(test_dataset)
test_df=test_df.drop(columns=['label_text'])
train_df, unlab_df= train_test_split(train_df, test_size=0.53, stratify=train_df['label'], random_state=42)
unlab_df.loc[:, 'label']=5
#For the first split, put the frac value below to be equal to 0.027
#For the second split, put the frac value below to be equal to 0.125
#For the third split, put the frac value below to be equal to 0.25
#For the fourth split, put the frac value below to be equal to 0.5
train_df=train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.5))
train_df.drop_duplicates(subset='text',inplace=True)
unlab_df.drop_duplicates(subset='text', inplace=True)
val_df.drop_duplicates(subset='text',inplace=True)
test_df.drop_duplicates(subset='text',inplace=True)
print(len(train_df))
print(len(unlab_df))
print(len(val_df))
print(len(test_df))

In [ ]:
def clean_text(text):
    soup=BeautifulSoup(text, "html.parser")
    text=re.sub(r"\[[^]]*\]" , '', soup.get_text())
    text=re.sub(r"[^A-Za-z0-9\s,']", '', text)
    return text

In [ ]:
train_df['cleaned']=train_df[train_df.columns[0]].apply(clean_text)
unlab_df['cleaned']=unlab_df[train_df.columns[0]].apply(clean_text)
val_df['cleaned']=val_df[train_df.columns[0]].apply(clean_text)
test_df['cleaned']=test_df[train_df.columns[0]].apply(clean_text)

In [ ]:
train_review=train_df['cleaned']
unlab_review=unlab_df['cleaned']
valX=val_df['cleaned']
testX=test_df['cleaned']
train_target=train_df[train_df.columns[1]]
valY=val_df[train_df.columns[1]]
testY=test_df[train_df.columns[1]]
unsup_target=unlab_df[train_df.columns[1]]

In [ ]:
tz=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
max_len= 64
X_train_encoded = tz.batch_encode_plus(train_review.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
X_val_encoded = tz.batch_encode_plus(valX.tolist(),
                                              padding='max_length',
                                              truncation=True, max_length=max_len,
                                              return_tensors='tf')
X_test_encoded = tz.batch_encode_plus(testX.tolist(),
                                              padding='max_length',
                                              truncation=True, max_length=max_len,
                                              return_tensors='tf')
X_unsup_encoded=tz.batch_encode_plus(unlab_review.tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

In [ ]:
TR1=X_train_encoded['input_ids']
TR2=X_train_encoded['attention_mask']
U1=X_unsup_encoded['input_ids']
U2=X_unsup_encoded['attention_mask']
V1=X_val_encoded['input_ids']
V2=X_val_encoded['attention_mask']
TS1=X_test_encoded['input_ids']
TS2=X_test_encoded['attention_mask']

In [ ]:
print(TR1.shape)
print(TR2.shape)
print(V1.shape)
print(V2.shape)
print(TS1.shape)
print(TS2.shape)
print(U1.shape)
print(U2.shape)
print(train_target.shape)
print(valY.shape)
print(testY.shape)
print(unsup_target.shape)

In [ ]:
trainY=tf.one_hot(train_target, depth=train_target.nunique())
valY=tf.one_hot(valY, depth=valY.nunique())
testY=tf.one_hot(testY, depth=testY.nunique())

In [ ]:
j=0 # the current number of iteration
iter=2 #the number of iterations for which the first dropout value will be applied, you change it according to the experiments in the pdf. If the dropout value is the same in all iterations, you just put the sae value twice below wehre you set dropout
total=0 #the sum of all iterations
history=[]
while U1.shape[0] != 0:
    config=BertConfig.from_pretrained('bert-base-uncased')
    config.num_labels=trainY.shape[1]
    if j < iter:
        config.hidden_dropout_prob=0.2
    elif j >= iter:
        config.hidden_dropout_prob=0.2
    model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
    num_epochs=100
    if trainY.shape[1] == 2:
        loss=tf_keras.losses.BinaryCrossentropy(from_logits=True)
        metric=tf_keras.metrics.BinaryAccuracy('accuracy')
    elif trainY.shape[1] == 5:
        loss=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
        metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
    if j == 0:
        initial_lr=1e-5
        optimizer=tf_keras.optimizers.Adam(learning_rate=initial_lr)
        callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
    if j > 0:
        lr=9e-6
        optimizer=tf_keras.optimizers.Adam(learning_rate=lr)
        callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    history.append(model.fit([TR1, TR2], trainY, validation_data=([V1, V2], valY), batch_size=32, epochs=num_epochs, shuffle=True, callbacks=[callback], verbose=1))
    y_pred=model.predict([U1, U2])
    if trainY.shape[1] == 2:
        probs=tf.reduce_max(tf.nn.sigmoid(y_pred['logits']), axis=1)
        preds=tf.one_hot(tf.argmax(tf.nn.sigmoid(y_pred['logits']), axis=1), depth=trainY.shape[1])
    elif trainY.shape[1] == 5:
        probs=tf.reduce_max(tf.nn.softmax(y_pred['logits'], axis=1), axis=1)
        preds=tf.one_hot(tf.argmax(tf.nn.softmax(y_pred['logits'], axis=1), axis=1), depth=trainY.shape[1])
    count=0
    indexes=[]
    p=0.9 #p, the threshold which the max predicted probability of a sample must overcome so that it be added to the training data
    for i in range(probs.shape[0]):
        if probs[i] >= p:
            indexes.append(i)
            count+=1
            TR1=tf.concat([TR1, tf.reshape(U1[i], (1, max_len))], axis=0)
            TR2=tf.concat([TR2, tf.reshape(U2[i], (1, max_len))], axis=0)
            trainY=tf.concat([trainY, tf.reshape(preds[i], (1, trainY.shape[1]))], axis=0)
    print(TR1.shape)
    print(TR2.shape)
    print(trainY.shape)
    if len(indexes) != 0:
        maskX=tf.ones(U1.shape[0], tf.bool)
        maskY=tf.ones(unsup_target.shape[0], tf.bool)
        indices_to_update=tf.expand_dims(indexes, axis=1)
        updatesX=tf.zeros_like(indexes, dtype=tf.bool)
        updatesY=tf.zeros_like(indexes, dtype=tf.bool)
        maskX=tf.tensor_scatter_nd_update(maskX, indices_to_update, updatesX)
        maskY=tf.tensor_scatter_nd_update(maskY, indices_to_update, updatesY)
        U1=tf.boolean_mask(U1, maskX)
        U2=tf.boolean_mask(U2, maskX)
        unsup_target=tf.boolean_mask(unsup_target, maskY)
        print(U1.shape)
        print(U2.shape)
        print(unsup_target.shape)
    print(count, "samples were added to the labelled set")
    total+=count
    j+=1
    if j==4:
        break

In [ ]:
test_loss, test_accuracy = model.evaluate([TS1, TS2], testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

In [ ]:
#HERE YOU RUN THE SUPERVISED MODEL FOR THE SAME DATA. IF YOU HAVE RAN THE SEMI SUPERVISED MODEL FIRST, THEN YOU HAVE TO RUN FIRST THE CELLS WHICH START WITH THE DEFINITION OF TRAIN REVIEW, TRAIN TARGET ETC UNTIL THE SEMI SUPERVISED MODEL.THEN RUN THIS CELL
num_epochs=100
config=BertConfig.from_pretrained('bert-base-uncased')
config.num_labels=trainY.shape[1]
config.hidden_dropout_prob=0.2
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
initial_lr=1e-5
optimizer=tf_keras.optimizers.Adam(learning_rate=initial_lr)
if trainY.shape[1] == 2:
    loss=tf_keras.losses.BinaryCrossentropy(from_logits=True)
    metric=tf_keras.metrics.BinaryAccuracy('accuracy')
elif trainY.shape[1] == 5:
    loss=tf_keras.losses.CategoricalCrossentropy(from_logits=True)
    metric=tf_keras.metrics.CategoricalAccuracy('accuracy')
callback=tf_keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
historySup=model.fit([TR1, TR2], trainY, validation_data=([V1, V2], valY), batch_size=32, epochs=num_epochs, shuffle=True, callbacks=[callback], verbose=1)

In [ ]:
test_loss, test_accuracy = model.evaluate([TS1, TS2], testY)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')